In [59]:
import json
import random

import numpy as np
import razdel
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import corpus_bleu

In [60]:
def read_gazeta_records(file_name, shuffle=False, sort_by_date=True):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle(records)
    return records

test_records = read_gazeta_records("../dataset/gazeta_test.jsonl")

### Lead-n

In [63]:
import razdel

def predict_lead(text, summary, n):
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    prediction = " ".join(sentences[:n])
    return prediction

### TextRank¶

In [64]:
from summa.summarizer import summarize

def predict_text_rank(text, headline, summary_part=0.1):
    return summarize(text, ratio=summary_part, language='russian').replace("\n", " ")

### LexRank

In [65]:
import lexrank
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS


def predict_lex_rank(text, summary, lxr, summary_size=3, threshold=None):
    sentences = [s.text for s in razdel.sentenize(text)]
    prediction = lxr.get_summary(sentences, summary_size=summary_size, threshold=threshold)
    prediction = " ".join(prediction)
    return prediction

In [54]:
df_gazeta = pd.read_csv('../outputs/gazeta_v1.csv')

In [56]:
df_gazeta = df_gazeta.loc[:, ~df_gazeta.columns.str.contains('^Unnamed')]

In [57]:
df_gazeta

,summary,title,text
0,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...
1,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...
2,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...
3,Действия американских ВС в Эстонии во время уч...,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...
4,Поправки в российский закон «О банкротстве» вс...,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...
...,...,...,...
6788,Первый пуск американской частной ракеты Alpha ...,СМИ назвали успехом запуск частной ракеты Alph...,Феерическим взрывом в атмосфере закончился пер...
6789,Городская агломерация Владивостока может дости...,На ВЭФ подписано соглашение о создании города ...,Глава Минвостокразвития Алексей Чекунков 31 ав...
6790,Из-за геополитической и экономической роли Рос...,Глава чешского МИДа призвал восстанавливать св...,После взрывов во Врбетице отношения между Чехи...
6791,Международная группа астрономов впервые наблюд...,Астрономы наблюдали вспышку сверхновой нового ...,Астрономы впервые наблюдали процесс вспышки св...


In [75]:
for index, row in df_gazeta.iterrows():
    headline = df_gazeta['title'][index]
    text = df_gazeta['text'][index]
    sentences = [s.text for s in razdel.sentenize(text)]
    lxr = LexRank(sentences, stopwords=STOPWORDS['ru'])
    df_gazeta.at[index, 'lead_1'] = predict_lead(text, headline, 1)
    df_gazeta.at[index, 'lead_2'] = predict_lead(text, headline, 2)
    df_gazeta.at[index, 'lead_3'] = predict_lead(text, headline, 3)
    df_gazeta.at[index, 'text_rank'] = predict_text_rank(text, headline, summary_part=0.1)
    df_gazeta.at[index, 'lex_rank'] = predict_lex_rank(text, headline, lxr, summary_size=3, threshold=None)
#     df_rbc.at[index, 'lead_3'] = predict_lead(text, headline, 3)

In [76]:
df_gazeta.head(3)

,summary,title,text,lex_rank,lead_1,lead_2,lead_3,text_rank
0,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие начали собираться в субботу с сам...,На этих выходных в Берлине прошли крупные акци...,На этих выходных в Берлине прошли крупные акци...,На этих выходных в Берлине прошли крупные акци...,"По его мнению, протестующие ранее пришли к рос..."
1,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,Именно с подачи президента США Дональда Трампа...,Высокопоставленная американская и израильская ...,Высокопоставленная американская и израильская ...,Высокопоставленная американская и израильская ...,Высокопоставленная американская и израильская ...
2,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Одна из руководителей Координационного совета ...,Одна из руководителей Координационного совета ...,Одна из руководителей Координационного совета ...,Одна из руководителей Координационного совета ...,Одна из руководителей Координационного совета ...


In [77]:
df_gazeta.to_csv('../outputs/gazeta_baseline.csv')

In [78]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../scripts/')
import pandas as pd
import evaluate

In [79]:
from evaluate import calculate_Bleu_Score, calculate_Rouge_Score, evalute_sample_from_dataset
from plot_eval import plot_rouge, plot_bleu

In [80]:
r_l1 = df_gazeta[['lead_1','title']].astype(str)
r_l2 = df_gazeta[['lead_2','title']].astype(str)
r_l3 = df_gazeta[['lead_3','title']].astype(str)
r_text_rank = df_gazeta[['text_rank','title']].astype(str)
r_lex_rank = df_gazeta[['lex_rank','title']].astype(str)

In [81]:
metrics = {}
metrics.update({'LEAD_1':r_l1})
metrics.update({'LEAD_2':r_l1})
metrics.update({'LEAD_3':r_l2})
metrics.update({'TextRank':r_text_rank})
metrics.update({'LexRank':r_lex_rank})

In [82]:
for name, metric in metrics.items():
    bleu_scores, rouge1_scores, rouge2_scores, rougeL_scores, rougeLsum_scores = evalute_sample_from_dataset(metric, 300)
    bs = [np.round(i*100, 1) for i in bleu_scores]
    r1 = [np.round(i*100, 1) for i in rouge1_scores]
    r2 = [np.round(i*100, 1) for i in rouge2_scores]
    r_l = [np.round(i*100, 1) for i in rougeL_scores]
    print("============Metric {}===========".format(name))
    print("============BLEU===========", "\n", bs)
    print("============ROUGE-1===========", "\n", r1)
    print("============ROUGE-2===========", "\n", r2)
    print("============ROUGE-L===========", "\n", r_l)

Final Blue Scores [[0.16000743 0.06260251 0.02987202]]
Final ROUGE1 Scores [[0.13805556 0.1035     0.11122222]]
Final ROUGE2 Scores [[0.03666667 0.02711111 0.02911111]]
Final ROUGEL Scores [[0.13388889 0.10116667 0.10826984]]
Final ROUGELsum Scores [[0.13388889 0.10116667 0.10826984]]
============Metric LEAD_1===========
============BLEU=========== 
 [array([16. ,  6.3,  3. ])]
============ROUGE-1=========== 
 [array([13.8, 10.4, 11.1])]
============ROUGE-2=========== 
 [array([3.7, 2.7, 2.9])]
============ROUGE-L=========== 
 [array([13.4, 10.1, 10.8])]
Final Blue Scores [[0.16000743 0.06260251 0.02987202]]
Final ROUGE1 Scores [[0.13805556 0.1035     0.11122222]]
Final ROUGE2 Scores [[0.03666667 0.02711111 0.02911111]]
Final ROUGEL Scores [[0.13388889 0.10116667 0.10826984]]
Final ROUGELsum Scores [[0.13388889 0.10116667 0.10826984]]
============Metric LEAD_2===========
============BLEU=========== 
 [array([16. ,  6.3,  3. ])]
============ROUGE-1=========== 
 [array([13.8, 10.4, 11.1]